In [1]:
# 1 加载必要的库

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
# 2 定义超参数

'''
电脑的性能有限，在训练的时候需要一批批的训练
将 60000 张图片分批次地放进模型中进行训练
BATCH_SIZE 代表每一批图片的数量
'''
BATCH_SIZE = 128 

'''
在训练模型的时候，是使用CPU还是GPU
'''
DEVICE = torch.device('cuda' if torch.cuda.is_available else 'cpu')

'''
60000 张图片训练几轮
'''
EPOCHS = 10

In [3]:
# 3 构建pipeline，对图像做处理

pipeline = transforms.Compose([
    transforms.ToTensor(), # 将图片转换成tensor
    transforms.Normalize((0.1307,),(0.3081,)), # 正则化防止过拟合, 参数是官网提供的
    # 降低模型复杂度
])

In [4]:
# 4 下载、加载数据

from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline) 

test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据集
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)


V:\Environment\Annaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# 拿到图片之后是二进制的数据流，是压缩过的，我们如何才能看到实际的图片呢？

# 插入代码，显示MNIST中的图片

with open('./data/MNIST/raw/train-images-idx3-ubyte', "rb") as f:
    file = f.read()

In [6]:
image1 = [int(str(item).encode('ascii'), 16) for item in file[16 : 16+784]]

In [7]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28, 28, 1)

print(image1_np.shape)

(28, 28, 1)


In [8]:
cv2.imwrite('digit.jpg', image1_np)

True

In [9]:
# 5 构建网络模型

class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        '''
        1代表灰度图片的通道(输入通道)
        10代表输出通道
        5代表卷积核的大小
        '''
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 3)
        '''
        输入通道为 20*10*10
        输出通道为 500
        '''
        self.fc1 = nn.Linear(20*10*10, 500)
        '''
        输入通道 500层
        输出通道 10 层，10个不同的数字
        '''
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        input_size = x.size(0) # batch_size *1*28*28
        x = self.conv1(x) # 输入：batch_size*1*28*28，输出 batch*10*24*24
        x = F.relu(x) # 保持shape不变，输出：batch*10*24*24
        x = F.max_pool2d(x, 2, 2) # 输入batch*10*24*24，输出batch*10*12*12
        
        x = self.conv2(x) # 输入batch*10*12*12 输出 batch*20*10*10
        x = F.relu(x) # 激活
        
        x = x.view(input_size, -1) # 拉伸，-1表示自动计算维度
        
        x = self.fc1(x) # 输入 batch*2000 输出：batch*500
        x = F.relu(x) # 保持 shape 不变
        
        x = self.fc2(x) # 输入 batch*500 输出： batch*10
        
        output = F.log_softmax(x, dim=1) # 计算分类后每个数字的概率值
        
        return output

In [10]:
# 6 定义优化器

model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [11]:
# 7 定义训练方法

def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, [data, target] in enumerate(train_loader):
        # 部署到devidce上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为 0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 反向传播
        loss.backward()
        # 参数优化
        if batch_index % 3000 == 0:
            print('Train epoch : {} \t Loss: {:6f}'.format(epoch, loss.item()))
        

In [14]:
# 8 定义测试方法

def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad(): # 不计算梯度，不进行反向传播
        for data, target in test_loader:
            # 部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1]
            # 累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test —— Average loss : {:.4f}, Accuracy: {:.3f}\n".format(test_loss, 100.0*correct/len(test_loader.dataset)))
        

In [15]:
# 9 调用 7 8
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train epoch : 1 	 Loss: 2.314903
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 2 	 Loss: 2.302280
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 3 	 Loss: 2.311625
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 4 	 Loss: 2.301925
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 5 	 Loss: 2.306779
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 6 	 Loss: 2.301175
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 7 	 Loss: 2.293839
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 8 	 Loss: 2.300429
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 9 	 Loss: 2.303289
Test —— Average loss : 0.0182, Accuracy: 5.300

Train epoch : 10 	 Loss: 2.297224
Test —— Average loss : 0.0182, Accuracy: 5.300

